1. In this part, you will obtain as much data as you can on the campaign contributions received by each candidate. This data is avaiable through the website https://www.opensecrets.org/. At the end of the project, your group will give a presentation of your findings.

Start by scraping the data from the summary page for Tennessee's 2nd District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.
The data that we want is contained in the "Total Raised and Spent" table.
Make a DataFrame showing, for each candidate:
the candidate's name
the candidate's party
state
district number
whether the candidate was an incumbent
whether the candidate won the race
the total amount raised by that candidate (as a numeric variable)
the total amount spent by the candidate (as a numeric variable)

In [275]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import re 

In [276]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'
response = requests.get(URL)
type(response)
response.status_code
#response.text
soup = BS(response.text)
#print(soup.prettify())


In [277]:
#found the table type in print(soup.prettify):
 #<table class="DataTable">
soup.find('table', attrs={'class' : 'DataTable'}) # does not need attr (but being more specific is helpful in attr)
table_html = str(soup.find('table', attrs={'class' : 'DataTable'}))
#just to show the table, not to interact with it yet
HTML(table_html)

Candidate,Raised,Spent,Cash on Hand,Last Report
Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020
Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020


In [278]:
#interact with the table
pd.read_html(str(soup.find('table', attrs={'class' : 'DataTable'})))[0] #first table

,Candidate,Raised,Spent,Cash on Hand,Last Report
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020


2. Once you have working code for Tennessee's 2nd District, expand on your code to capture all of Tennessee's districts.

In [279]:
URL = 'https://www.factmonster.com/us/postal-information/state-abbreviations-and-state-postal-codes'
response = requests.get(URL)
type(response)
response.status_code
#response.text
soup = BS(response.text)
#print(soup.prettify())

In [280]:
#now find a list of the states and turn it into a DF
soup.find('table')
table_html = str(soup.find('table'))
#just to show the table, not to interact with it yet
HTML(table_html)


State/District,Abbreviation,Postal Code
Alabama,Ala.,AL
Alaska,Alaska,AK
Arizona,Ariz.,AZ
Arkansas,Ark.,AR
California,Calif.,CA
Colorado,Colo.,CO
Connecticut,Conn.,CT
Delaware,Del.,DE
District of Columbia,D.C.,DC
Florida,Fla.,FL


In [281]:
#interact with the table
states_list = pd.read_html(str(soup.find('table')))[0] #first table
states_list = states_list.rename(columns = {'State/District' : 'state'})
states_list.head(2)

,state,Abbreviation,Postal Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK


now, find the list of the states with their districts from https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120
...and turn the table into a DF


In [282]:
URL = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'
response = requests.get(URL)
type(response)
response.status_code
#response.text
soup = BS(response.text)
#print(soup.prettify())
soup.find('table')
table_html = str(soup.find('table'))
#just to show the table, not to interact with it yet
HTML(table_html)

state,representatives
Alabama,7
Alaska,1
Arizona,9
Arkansas,4
California,53
Colorado,7
Connecticut,5
Delaware,1
Florida,27
Georgia,14


In [283]:
states_list.head(2)

,state,Abbreviation,Postal Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK


In [284]:
#interact with the table
district_numbers = pd.read_html(str(soup.find('table')))[0] #first table
district_numbers.head(2)

,state,representatives
0,Alabama,7
1,Alaska,1


In [285]:
#merge the states_list and district_numbers
states_and_districts = pd.merge(states_list, district_numbers)
states_and_districts.head(10)

,state,Abbreviation,Postal Code,representatives
0,Alabama,Ala.,AL,7
1,Alaska,Alaska,AK,1
2,Arizona,Ariz.,AZ,9
3,Arkansas,Ark.,AR,4
4,California,Calif.,CA,53
5,Colorado,Colo.,CO,7
6,Connecticut,Conn.,CT,5
7,Delaware,Del.,DE,1
8,Florida,Fla.,FL,27
9,Georgia,Ga.,GA,14


In [286]:
#change the column name for future use
states_and_districts = states_and_districts.rename(columns = {'Postal Code': 'state_abbr'}) #state abbreviation
states_and_districts.head(10)

,state,Abbreviation,state_abbr,representatives
0,Alabama,Ala.,AL,7
1,Alaska,Alaska,AK,1
2,Arizona,Ariz.,AZ,9
3,Arkansas,Ark.,AR,4
4,California,Calif.,CA,53
5,Colorado,Colo.,CO,7
6,Connecticut,Conn.,CT,5
7,Delaware,Del.,DE,1
8,Florida,Fla.,FL,27
9,Georgia,Ga.,GA,14


In [287]:
TN_state = states_and_districts[states_and_districts['state_abbr'] == 'TN']
TN_state

,state,Abbreviation,state_abbr,representatives
41,Tennessee,Tenn.,TN,9


In [288]:
#base url to which nine of two digit numbers are to be added for pulling the first table from each
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN'

#create a list of district codes
district_codes = []
for district in range(1, int(TN_state['representatives']) + 1): #+1 includes the last district as well
    if district < 10:
        district_codes.append('0'+ str(district)) #otherwise 0+9 = 9 if treating as digits
    else:
        district_codes.append(str(district)) 
print(district_codes)

#create a list of URLs
URL_list = []
#apped district codes to URL list items
for code in district_codes:
    URL_list.append(URL + code)
print(URL_list)


#pull out each table data and add each to a list containing all tables
table_list = []
for URL in URL_list:
    response = requests.get(URL)
    soup = BS(response.text)
    TN_table = pd.read_html(str(soup.find('table', attrs = {'class' : 'DataTable'})))[0]
    table_list.append(TN_table)
table_list

['01', '02', '03', '04', '05', '06', '07', '08', '09']
['https://www.opensecrets.org/races/summary?cycle=2020&id=TN01', 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02', 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN03', 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN04', 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN05', 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN06', 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN07', 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN08', 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN09']


[                         Candidate      Raised       Spent Cash on Hand  \
 0  Diana Harshbarger (R)  • Winner  $2,126,946  $1,869,100     $257,846   
 1      Blair Nicole Walsingham (D)    $140,209    $134,995       $5,215   
 
   Last Report  
 0  12/31/2020  
 1  10/14/2020  ,
                                 Candidate      Raised     Spent Cash on Hand  \
 0  Tim Burchett (R)  • Incumbent • Winner  $1,336,276  $878,488     $593,678   
 1                         Renee Hoyos (D)    $812,784  $816,793         $210   
 
   Last Report  
 0  12/31/2020  
 1  12/31/2020  ,
                                      Candidate      Raised     Spent  \
 0  Chuck Fleischmann (R)  • Incumbent • Winner  $1,051,653  $381,411   
 1                               Meg Gorman (D)     $85,843   $77,760   
 
   Cash on Hand Last Report  
 0   $1,880,341  12/31/2020  
 1       $8,083  12/31/2020  ,
                                     Candidate    Raised     Spent  \
 0  Scott Desjarlais (R)  • Incumbent •

In [289]:
base_table = table_list.pop(0)
base_table

,Candidate,Raised,Spent,Cash on Hand,Last Report
0,Diana Harshbarger (R) • Winner,"$2,126,946","$1,869,100","$257,846",12/31/2020
1,Blair Nicole Walsingham (D),"$140,209","$134,995","$5,215",10/14/2020


In [290]:
#concatenate all the tables (each is a DF) 
# concat is used for when the values for the columns are different. merge needs similar values
for table in table_list:
    all_TN_tables = pd.concat([base_table, table])
all_TN_tables

,Candidate,Raised,Spent,Cash on Hand,Last Report
0,Diana Harshbarger (R) • Winner,"$2,126,946","$1,869,100","$257,846",12/31/2020
1,Blair Nicole Walsingham (D),"$140,209","$134,995","$5,215",10/14/2020
0,Steve Cohen (D) • Incumbent • Winner,"$607,570","$457,311","$1,216,119",12/31/2020
1,Charlotte Bergmann (R),"$99,147","$100,532","$2,566",12/31/2020
2,Dennis Clark (I),"$9,991","$9,919",$40,11/23/2020


3. Once you have working code for all of Tennessee's districts, expand on it to capture all states and districts. The number of representatives each state has can be found in a table on this page: https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120

In [291]:
states_and_districts.head()

,state,Abbreviation,state_abbr,representatives
0,Alabama,Ala.,AL,7
1,Alaska,Alaska,AK,1
2,Arizona,Ariz.,AZ,9
3,Arkansas,Ark.,AR,4
4,California,Calif.,CA,53


In [292]:
#base url for all states (lacks state abbr and hte district codes)

#each row is related to a state
district_codes = {} #put here, not after for loop so it does not empty after each iteration
for index, row in states_and_districts.iterrows(): #iterrow gives a tuple of index and row content
    district_codes[row[2]] = []
    for district in range(1, int(row[3])+1): 
        if district < 10:
            district_codes[row[2]].append(row[2] + '0' + str(district))
        else:
            district_codes[row[2]].append(row[2] + str(district)) 
#print(district_codes) # a dict 

In [293]:
URL_list = []
for key, item in district_codes.items(): 
  for value in item:
    #print(value)
    URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=' #not corrupted here
    #print(URL)
    URL_list.append(URL + value) #URL here is not fresh anymore! So I bring it inside
    #print(URL)
#print(URL_list)

In [366]:
table_list = []
for URL in URL_list:
    response = requests.get(URL)
    soup = BS(response.text)
    state_table = pd.read_html(str(soup.find('table', attrs = {'class' : 'DataTable'})))[0] #make DF
    state_table['state_info'] = URL[-4:]
   
    table_list.append(state_table)
#print(table_list)
#print(table_list)
#output: tables (DFs) for all URLs that need to be concatinated. district_codes

In [402]:
# concatination needs a base DF:  
base_table= table_list.pop(0)
for table in table_list:
     base_table = pd.concat([base_table, table])
#print(base_table)


In [403]:
base_table = base_table.reset_index(drop = True)
#base_table = base_table.assign(Name = '', Party = '', Winner = '', Incumbent = '')
base_table.head(20)

,Candidate,Raised,Spent,Cash on Hand,Last Report,state_info
0,Mike D Rogers (R) • Incumbent • Winner,"$1,193,111","$1,218,564","$502,234",12/31/2020,AL03
1,Adia Winfrey (D),"$50,273","$40,971",$43,12/31/2020,AL03
2,Robert B Aderholt (R) • Incumbent • Winner,"$1,255,076","$1,323,812","$647,004",12/31/2020,AL04
3,Rick Neighbors (D),"$46,068","$45,034","$1,034",12/31/2020,AL04
4,Mo Brooks (R) • Incumbent • Winner,"$655,365","$210,045","$1,137,501",12/31/2020,AL05
5,Gary Palmer (R) • Incumbent • Winner,"$907,219","$909,082","$370,688",12/31/2020,AL06
6,Terri Sewell (D) • Incumbent • Winner,"$2,168,165","$1,495,957","$2,243,480",12/31/2020,AL07
7,Don Young (R) • Incumbent • Winner,"$1,922,901","$1,790,448","$249,173",12/31/2020,AK01
8,Alyse Galvin (I),"$5,178,452","$5,088,103","$96,594",12/31/2020,AK01
9,Tom O'Halleran (D) • Incumbent • Winner,"$3,381,353","$3,324,138","$63,147",12/31/2020,AZ01


In [408]:
#clean the dataframe
base_table['winner'] = base_table['Candidate'].str.extract('([wW]inner)')
base_table['party'] = base_table['Candidate'].str.extract('\((\w)\)')
base_table['incumbent'] = base_table['Candidate'].str.extract('(incumbent)', flags = re.I)
base_table['name'] = base_table['Candidate'].str.extract('(.+)\(')
base_table['state-district'] = base_table['state_info'].str.extract('(\d{2})')
base_table['state-district'].astype(int)
base_table['state-code'] = base_table['state_info'].str.extract('(\D{2})')
base_table[['Raised', 'Spent', 'Cash on Hand']] = base_table[['Raised', 'Spent', 'Cash on Hand']].replace('[\$,]', '', regex = True).astype(int)
base_table['Last Report'] = pd.to_datetime(base_table['Last Report'])
base_table.drop(columns = ['state_info'])
final_table = base_table[['name', 'state-code', 'state-district', 'winner', 'party', 'incumbent', 'Raised', 'Spent', 'Cash on Hand', 'Last Report']]
final_table

,name,state-code,state-district,winner,party,incumbent,Raised,Spent,Cash on Hand,Last Report
0,Mike D Rogers,AL,03,Winner,R,Incumbent,1193111,1218564,502234,2020-12-31
1,Adia Winfrey,AL,03,NaN,D,NaN,50273,40971,43,2020-12-31
2,Robert B Aderholt,AL,04,Winner,R,Incumbent,1255076,1323812,647004,2020-12-31
3,Rick Neighbors,AL,04,NaN,D,NaN,46068,45034,1034,2020-12-31
4,Mo Brooks,AL,05,Winner,R,Incumbent,655365,210045,1137501,2020-12-31
...,...,...,...,...,...,...,...,...,...,...
885,Tricia Zunker,WI,07,NaN,D,NaN,1261957,1232690,29267,2020-12-31
886,Mike Gallagher,WI,08,Winner,R,Incumbent,3202905,2841801,1169362,2020-12-31
887,Amanda Stuck,WI,08,NaN,D,NaN,416978,399916,2165,2020-12-31
888,Liz Cheney,WY,01,Winner,R,Incumbent,3003883,3060167,153567,2020-12-31


In [409]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            890 non-null    object        
 1   state-code      890 non-null    object        
 2   state-district  890 non-null    object        
 3   winner          429 non-null    object        
 4   party           890 non-null    object        
 5   incumbent       386 non-null    object        
 6   Raised          890 non-null    int64         
 7   Spent           890 non-null    int64         
 8   Cash on Hand    890 non-null    int64         
 9   Last Report     890 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 69.7+ KB


4. Using your scraped data, investigates different relationships between candidates and the amount of money they raised. Here are some suggestions to get you started, but feel free to pose you own questions or do additional exploration:
a. How often does the candidate who raised more money win a race?
b. How often does the candidate who spent more money win a race?
c. Does the difference between either money raised or money spent seem to influence the likelihood of a candidate winning a race?
d. How often does the incumbent candidate win a race?
e. Can you detect any relationship between amount of money raised and the incumbent status of a candidate?

In [421]:
print(final_table['party'].value_counts())
final_table.groupby('winner')['party'].value_counts()

D    422
R    386
I     46
L     21
3     15
Name: party, dtype: int64


winner  party
Winner  D        221
        R        208
Name: party, dtype: int64